In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from tensorflow.keras import layers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

In [ ]:
from transformers import RobertaTokenizer, TFRobertaModel

# model = tf.keras.models.load_model('../input/roberta-base-loading/model-RoBERTa-base')
tokenizer = RobertaTokenizer.from_pretrained('../input/roberta-base-loading/tokenizer')
transformer = TFRobertaModel.from_pretrained('../input/roberta-base-loading/transformer')

input_ids_in = tf.keras.layers.Input(shape=(512,) ,name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(512,), name='masked_token', dtype='int32')

embedding_layer = transformer(input_ids_in, attention_mask=input_masks_in)[0]
X = layers.Bidirectional(layers.GRU(512, return_sequences=True))(embedding_layer)
X = layers.Bidirectional(layers.GRU(128))(X)
# X = layers.Bidirectional(layers.GRU(128))(embedding_layer)
X = layers.Dense(256, activation='relu')(X)
X = layers.Dense(64, activation='relu')(X)
X = layers.Dense(1, activation=None)(X)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

model.load_weights('../input/robertabase-augmented-data-model/saved-model-RoBERTa-base')

In [ ]:
# train_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
sample_submission = pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')

In [ ]:
inputs = tokenizer(list(test_df['excerpt'].values), return_tensors="tf",padding="max_length")

In [ ]:
test_preds = model.predict([inputs['input_ids'],inputs['attention_mask']])

In [ ]:
# test_preds
sample_submission['id'] = test_df['id']
sample_submission['target'] = test_preds

In [ ]:
sample_submission.to_csv('submission.csv',index=False)